In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
%load_ext kedro.ipython

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


In [3]:
movies_metadata = catalog.load("movies_metadata")
credits = catalog.load("credits")
ratings = catalog.load("ratings")


[10/27/25 20:10:05] INFO     Loading data from movies_metadata (CSVDataset)...                 ]8;id=171776;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=602470;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

[10/27/25 20:10:06] INFO     Loading data from credits (CSVDataset)...                         ]8;id=727787;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=261762;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

[10/27/25 20:10:07] INFO     Loading data from ratings (CSVDataset)...                         ]8;id=378528;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=550888;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

In [5]:
print(movies_metadata.head())
print(credits.head())
print(ratings.head())

   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497         

In [12]:
# Seleccionar solo columnas numéricas
numeric_cols = movies_metadata.select_dtypes(include=['float64', 'int64'])

# Calcular correlación con vote_count
correlations = numeric_cols.corr()['vote_count'].sort_values(ascending=False)
print(correlations)


vote_count        1.000000
revenue           0.936532
budget            0.861526
popularity        0.673361
runtime           0.223522
vote_average      0.143811
original_title   -0.034585
Name: vote_count, dtype: float64


In [21]:
# Mostrar todas las columnas
print(movies_metadata.columns.tolist())


['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count']


In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Cargar dataset
movies_metadata = catalog.load("movies_metadata")

# Convertir columnas numéricas
numeric_cols = ['revenue', 'budget', 'popularity', 'runtime', 'vote_average', 'vote_count']
for c in numeric_cols:
    movies_metadata[c] = pd.to_numeric(movies_metadata[c], errors='coerce')

# Eliminar filas con nulos
movies_metadata = movies_metadata.dropna(subset=numeric_cols + ['status'])

# Convertir 'status' en variables dummy
status_dummies = pd.get_dummies(movies_metadata['status'], drop_first=True)

# Variables predictoras
X = movies_metadata[['revenue', 'budget', 'popularity', 'runtime', 'vote_average']]
X = pd.concat([X, status_dummies], axis=1)

# Variable objetivo
y = movies_metadata['vote_count']

# Dividir en training y testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Crear y entrenar el modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Predicciones
y_pred = model.predict(X_test)

# Métricas
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_test, y_pred)

print(f"Regresión Múltiple: vote_count ~ revenue + budget + popularity + runtime + vote_average + status")
print("="*50)
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")
print("="*50)

# Coeficientes
feature_names = X.columns
print("\nCoeficientes del modelo:")
for i, coef in enumerate(model.coef_):
    print(f"  {feature_names[i]}: {coef:.4e}")
print(f"  Intercepto: {model.intercept_:.4f}")

# Interpretación del R²
if r2 > 0.7:
    print("\n Modelo con buen ajuste (R² > 0.7)")
elif r2 > 0.5:
    print("\n Modelo con ajuste moderado (R² > 0.5)")
else:
    print("\n Modelo con ajuste pobre (R² < 0.5)")


[10/27/25 20:30:55] INFO     Loading data from movies_metadata (CSVDataset)...                 ]8;id=34793;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=240398;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

Regresión Múltiple: vote_count ~ revenue + budget + popularity + runtime + vote_average + status
Mean Squared Error (MSE): 79448.7927
Mean Absolute Error (MAE): 73.6543
Root Mean Squared Error (RMSE): 281.8666
R² Score: 0.7047

Coeficientes del modelo:
  revenue: 5.0397e-06
  budget: 2.8860e-06
  popularity: 1.2424e+01
  runtime: 6.3624e-02
  vote_average: 9.0669e+00
  In Production: 2.1837e+01
  Planned: -2.5933e+00
  Post Production: 3.1073e+00
  Released: 1.1924e+01
  Rumored: 7.2531e+00
  Intercepto: -64.7498

 Modelo con buen ajuste (R² > 0.7)


Conclusiones

Evaluación General

R² = 0.7047 → Modelo con buen ajuste

Explica el 70.47% de la variabilidad del número de votos (vote_count)

